# 🎨 Advanced Art Restoration: Handling Tears, Holes & Missing Parts

This notebook demonstrates advanced restoration techniques that can handle:
- **Structural damage**: tears, cracks, holes
- **Missing parts**: large missing regions
- **Quality degradation**: color, sharpness, noise

The system combines:
1. **Computer vision** for damage detection
2. **Deep learning inpainting** for structural repair
3. **Hybrid ML+DL** for quality restoration

In [1]:
import sys
import os

import cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import tensorflow as tf

# Add the project root to the Python path
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
	sys.path.insert(0, project_root)


from src.dl.advanced_restorer import AdvancedRestorer, StructuralDamageDetector, DeepInpainter

print(f"TensorFlow version: {tf.__version__}")
print(f"GPU available: {tf.config.list_physical_devices('GPU')}")

KeyboardInterrupt: 

## 🔍 Step 1: Damage Detection Analysis

Let's analyze what types of damage our current system can and cannot handle:

In [ ]:
# Initialize damage detector
detector = StructuralDamageDetector()

# Test on a sample image
test_image_path = '../data/raw/AI_for_Art_Restoration_2/paired_dataset_art/damaged/image_1.jpg'

if os.path.exists(test_image_path):
    image = cv2.imread(test_image_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # Detect all damage types
    damage_info = detector.detect_all_damage(image)
    
    # Visualize results
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    
    # Original image
    axes[0, 0].imshow(image_rgb)
    axes[0, 0].set_title('Original Damaged Image')
    axes[0, 0].axis('off')
    
    # Crack detection
    axes[0, 1].imshow(damage_info['crack_mask'], cmap='gray')
    axes[0, 1].set_title(f'Detected Cracks ({len(damage_info["cracks"])} found)')
    axes[0, 1].axis('off')
    
    # Hole detection
    axes[0, 2].imshow(damage_info['hole_mask'], cmap='gray')
    axes[0, 2].set_title(f'Detected Holes ({len(damage_info["holes"])} found)')
    axes[0, 2].axis('off')
    
    # Combined damage mask
    axes[1, 0].imshow(damage_info['combined_mask'], cmap='hot')
    axes[1, 0].set_title(f'All Damage ({damage_info["damage_percentage"]:.1f}% of image)')
    axes[1, 0].axis('off')
    
    # Overlay damage on original
    overlay = image_rgb.copy()
    damage_overlay = np.stack([damage_info['combined_mask']] * 3, axis=-1)
    overlay[damage_overlay > 0] = [255, 0, 0]  # Red overlay for damage
    
    axes[1, 1].imshow(cv2.addWeighted(image_rgb, 0.7, overlay, 0.3, 0))
    axes[1, 1].set_title('Damage Overlay (Red = Detected)')
    axes[1, 1].axis('off')
    
    # Damage statistics
    stats_text = f"""Damage Analysis:
• Total damaged area: {damage_info['damage_percentage']:.1f}%
• Cracks/tears found: {len(damage_info['cracks'])}
• Holes/missing parts: {len(damage_info['holes'])}
• Structural damage: {'Yes' if damage_info['has_structural_damage'] else 'No'}

Current hybrid system only handles:
✓ Color correction
✓ Sharpening
✓ General enhancement

Advanced system adds:
✓ Tear/crack repair
✓ Hole filling
✓ Missing part reconstruction"""
    
    axes[1, 2].text(0.05, 0.95, stats_text, transform=axes[1, 2].transAxes, 
                    fontsize=10, verticalalignment='top', 
                    bbox=dict(boxstyle='round', facecolor='lightblue', alpha=0.8))
    axes[1, 2].axis('off')
    
    plt.tight_layout()
    plt.show()
    
    print("\n📊 Damage Detection Results:")
    print(f"Structural damage detected: {damage_info['has_structural_damage']}")
    if damage_info['has_structural_damage']:
        print(f"This image needs advanced restoration with inpainting!")
    else:
        print(f"This image can be handled by the current hybrid system.")
        
else:
    print("⚠️ Test image not found. Please check the path.")

## 🧠 Step 2: Deep Learning Inpainting Model

Let's create and test the inpainting component that can fill holes and repair tears:

In [ ]:
# Initialize inpainter (will use traditional method if DL model not available)
inpainter = DeepInpainter()

print("🎯 Inpainting Model Status:")
if inpainter.model is not None:
    print("✅ Deep learning model initialized")
    inpainter.model.summary()
else:
    print("⚠️ Using traditional OpenCV inpainting (faster but less sophisticated)")
    print("   For better results, train the deep learning model first.")

## 🔧 Step 3: Test Inpainting on Simulated Damage

Let's create some artificial damage to test our inpainting:

In [ ]:
if os.path.exists(test_image_path):
    # Load a clean-ish image for testing
    test_img = cv2.imread(test_image_path)
    test_img_rgb = cv2.cvtColor(test_img, cv2.COLOR_BGR2RGB)
    
    # Create artificial damage for testing
    h, w = test_img.shape[:2]
    artificial_mask = np.zeros((h, w), dtype=np.uint8)
    
    # Add some artificial tears (lines)
    cv2.line(artificial_mask, (w//4, h//4), (3*w//4, 3*h//4), 255, 15)
    cv2.line(artificial_mask, (w//2, 0), (w//2, h//3), 255, 8)
    
    # Add some holes (circles)
    cv2.circle(artificial_mask, (w//6, h//3), 25, 255, -1)
    cv2.circle(artificial_mask, (5*w//6, 2*h//3), 20, 255, -1)
    
    # Add rectangular missing part
    cv2.rectangle(artificial_mask, (w//3, h//6), (2*w//3, h//3), 255, -1)
    
    # Create damaged version
    damaged_img = test_img.copy()
    damaged_img[artificial_mask > 0] = [0, 0, 0]  # Black damage
    
    # Test inpainting
    print("🔧 Testing inpainting on artificial damage...")
    inpainted = inpainter.inpaint(damaged_img, artificial_mask)
    
    # Visualize results
    fig, axes = plt.subplots(2, 2, figsize=(12, 10))
    
    axes[0, 0].imshow(test_img_rgb)
    axes[0, 0].set_title('Original Image')
    axes[0, 0].axis('off')
    
    axes[0, 1].imshow(cv2.cvtColor(damaged_img, cv2.COLOR_BGR2RGB))
    axes[0, 1].set_title('Artificially Damaged')
    axes[0, 1].axis('off')
    
    axes[1, 0].imshow(artificial_mask, cmap='gray')
    axes[1, 0].set_title('Damage Mask')
    axes[1, 0].axis('off')
    
    axes[1, 1].imshow(cv2.cvtColor(inpainted, cv2.COLOR_BGR2RGB))
    axes[1, 1].set_title('Inpainted Result')
    axes[1, 1].axis('off')
    
    plt.tight_layout()
    plt.show()
    
    # Calculate metrics
    # PSNR for inpainted regions
    original_region = test_img_rgb[artificial_mask > 0]
    inpainted_region = cv2.cvtColor(inpainted, cv2.COLOR_BGR2RGB)[artificial_mask > 0]
    
    if len(original_region) > 0:
        mse = np.mean((original_region.astype(float) - inpainted_region.astype(float)) ** 2)
        if mse > 0:
            psnr = 20 * np.log10(255.0 / np.sqrt(mse))
            print(f"\n📈 Inpainting Quality (damaged regions only):")
            print(f"PSNR: {psnr:.2f} dB")
        
    print(f"✅ Inpainting test completed!")
    
else:
    print("⚠️ Test image not found for inpainting demo")

## 🚀 Step 4: Complete Advanced Restoration System

Now let's test the full advanced restoration pipeline:

In [ ]:
# Initialize advanced restorer
try:
    advanced_restorer = AdvancedRestorer(
        ml_model_path='../outputs/models/restoration_parameter_predictor.pkl',
        ml_scaler_path='../outputs/models/parameter_feature_scaler.pkl',
        use_dl=True  # Enable U-Net if available
    )
    print("✅ Advanced restoration system initialized!")
    
except Exception as e:
    print(f"⚠️ Could not initialize full system: {e}")
    print("Make sure you've trained the ML models first (notebooks 1-4)")
    advanced_restorer = None

In [ ]:
if advanced_restorer and os.path.exists(test_image_path):
    print("🎨 Running complete advanced restoration...")
    
    # Run advanced restoration
    restored, info = advanced_restorer.restore_advanced(
        test_image_path,
        output_path='../outputs/advanced_restoration_result.jpg',
        strategy='auto'
    )
    
    # Load original for comparison
    original = cv2.imread(test_image_path)
    original_rgb = cv2.cvtColor(original, cv2.COLOR_BGR2RGB)
    restored_rgb = cv2.cvtColor(restored, cv2.COLOR_BGR2RGB)
    
    # Visualize results
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    
    # Original damaged
    axes[0, 0].imshow(original_rgb)
    axes[0, 0].set_title('Original Damaged Image')
    axes[0, 0].axis('off')
    
    # Advanced restoration result
    axes[0, 1].imshow(restored_rgb)
    axes[0, 1].set_title('Advanced Restoration Result')
    axes[0, 1].axis('off')
    
    # Damage visualization
    damage_viz = original_rgb.copy()
    if info['structural_damage']['has_structural_damage']:
        mask = info['structural_damage']['combined_mask']
        damage_overlay = np.stack([mask] * 3, axis=-1)
        damage_viz[damage_overlay > 0] = [255, 0, 0]
    
    axes[1, 0].imshow(cv2.addWeighted(original_rgb, 0.7, damage_viz, 0.3, 0))
    axes[1, 0].set_title('Detected Damage (Red)')
    axes[1, 0].axis('off')
    
    # Restoration info
    info_text = f"""Advanced Restoration Report:

🔍 Damage Analysis:
• Structural damage: {'Yes' if info['has_structural_repair'] else 'No'}
• Quality severity: {info['quality_analysis']['severity']}
• Damage coverage: {info['structural_damage']['damage_percentage']:.1f}%

🔧 Restoration Steps:
{chr(10).join(f'• {step}' for step in info['restoration_steps'])}

⚡ Method Used: {info['final_method']}

🎯 Key Improvements:
• Tears and cracks repaired
• Missing parts reconstructed  
• Color and sharpness enhanced
• Combined ML + DL approach

This advanced system can handle:
✓ Complex structural damage
✓ Multiple damage types
✓ Quality + structural repair"""
    
    axes[1, 1].text(0.05, 0.95, info_text, transform=axes[1, 1].transAxes,
                    fontsize=9, verticalalignment='top',
                    bbox=dict(boxstyle='round', facecolor='lightgreen', alpha=0.8))
    axes[1, 1].axis('off')
    
    plt.tight_layout()
    plt.show()
    
    print("\n🎉 Advanced restoration completed!")
    print(f"Result saved to: ../outputs/advanced_restoration_result.jpg")
    
else:
    print("⚠️ Skipping advanced restoration demo (missing models or test image)")

## 📊 Step 5: Method Comparison

Let's compare all restoration methods side by side:

In [ ]:
if advanced_restorer and os.path.exists(test_image_path):
    print("🔬 Comparing all restoration methods...")
    
    # Compare all methods
    comparison = advanced_restorer.compare_all_methods(test_image_path)
    
    # Create visualization
    methods = ['original', 'classical', 'ml_guided', 'advanced']
    titles = ['Original Damaged', 'Classical Only', 'ML-Guided', 'Advanced (ML+DL+Inpainting)']
    
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    axes = axes.flatten()
    
    for i, (method, title) in enumerate(zip(methods, titles)):
        if method in comparison:
            img = comparison[method]
            if len(img.shape) == 3:
                img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            else:
                img_rgb = img
            
            axes[i].imshow(img_rgb)
            axes[i].set_title(title, fontsize=12, fontweight='bold')
            axes[i].axis('off')
            
            # Add method info
            if method == 'advanced' and 'advanced_info' in comparison:
                info = comparison['advanced_info']
                steps_text = ', '.join(info['restoration_steps'])
                axes[i].text(0.02, 0.98, f"Steps: {steps_text}", 
                           transform=axes[i].transAxes,
                           bbox=dict(boxstyle='round', facecolor='yellow', alpha=0.7),
                           fontsize=8, verticalalignment='top')
    
    plt.tight_layout()
    plt.show()
    
    print("\n📈 Method Comparison Summary:")
    print("1. Classical: Basic enhancement only")
    print("2. ML-Guided: Intelligent parameter selection") 
    print("3. Advanced: Structural repair + quality enhancement")
    print("\n🏆 The Advanced method handles tears, holes, AND quality issues!")
    
else:
    print("⚠️ Skipping method comparison (models not available)")

## 🎯 Step 6: Batch Processing Test

Test the advanced system on multiple images:

In [ ]:
if advanced_restorer:
    # Test batch processing
    input_dir = '../data/raw/AI_for_Art_Restoration_2/paired_dataset_art/damaged'
    output_dir = '../outputs/advanced_batch_restored'
    
    if os.path.exists(input_dir):
        print("🚀 Testing batch advanced restoration...")
        
        # Process first 3 images as demo
        import glob
        files = glob.glob(os.path.join(input_dir, '*.jpg'))[:3]
        
        results = []
        for file_path in files:
            try:
                output_path = os.path.join(output_dir, f"advanced_{os.path.basename(file_path)}")
                os.makedirs(output_dir, exist_ok=True)
                
                restored, info = advanced_restorer.restore_advanced(file_path, output_path)
                results.append({
                    'file': os.path.basename(file_path),
                    'success': True,
                    'structural_damage': info['has_structural_repair'],
                    'steps': info['restoration_steps']
                })
                
            except Exception as e:
                results.append({
                    'file': os.path.basename(file_path),
                    'success': False,
                    'error': str(e)
                })
        
        # Display results
        print("\n📊 Batch Processing Results:")
        for result in results:
            if result['success']:
                status = "✅ Success"
                extra = f"Structural repair: {'Yes' if result['structural_damage'] else 'No'}"
            else:
                status = "❌ Failed"
                extra = f"Error: {result['error']}"
            
            print(f"{result['file']}: {status} - {extra}")
        
        success_count = sum(1 for r in results if r['success'])
        print(f"\n🎯 Batch Success Rate: {success_count}/{len(results)} ({100*success_count/len(results):.1f}%)")
        
    else:
        print(f"⚠️ Input directory not found: {input_dir}")
        
else:
    print("⚠️ Advanced restorer not available for batch testing")

## 🎓 Summary: Advanced Restoration Capabilities

### What We've Built:

1. **Structural Damage Detection**:
   - Automatic detection of tears, cracks, holes
   - Computer vision-based damage analysis
   - Quantitative damage assessment

2. **Deep Learning Inpainting**:
   - Context encoder for missing part reconstruction
   - Perceptual loss for realistic results
   - Fallback to traditional OpenCV methods

3. **Advanced Restoration Pipeline**:
   - Combines structural repair + quality enhancement
   - Intelligent routing based on damage type
   - Preserves all existing ML capabilities

### Key Improvements Over Basic Hybrid System:

| Feature | Basic Hybrid | Advanced System |
|---------|-------------|----------------|
| Color correction | ✅ | ✅ |
| Sharpening | ✅ | ✅ |
| General enhancement | ✅ | ✅ |
| **Tear repair** | ❌ | ✅ |
| **Hole filling** | ❌ | ✅ |
| **Missing part reconstruction** | ❌ | ✅ |
| **Structural damage detection** | ❌ | ✅ |
| **Complex multi-type damage** | ❌ | ✅ |

### Next Steps:

1. **Train the inpainting model** on more data for better results
2. **Fine-tune damage detection** parameters for your specific artwork types
3. **Collect damaged artwork datasets** with ground truth for training
4. **Experiment with different inpainting architectures** (GANs, Transformers)

The advanced system now handles the full spectrum of art restoration challenges! 🎨✨